<a href="https://colab.research.google.com/github/azalbulla/IT3/blob/main/it3medvedeva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Игра для девочек онлайн

**Сюжет:** Дарья, студентка университета, стоит перед сложным выбором в пятничный вечер. Два парня приглашают её на свидание, а внутренний голос предлагает остаться дома.

**Три пути:**
1. **Алекс** - надёжный IT-специалист, который давно влюблён в Дарью. Свидание в уютном кафе, глубокие разговоры, постепенное развитие чувств.
2. **Максим** - харизматичный музыкант, в которого Дарья влюблена. Концерт, страсть, эмоциональные качели, но нестабильные отношения.
3. **Сама с собой** - вечер самопознания. Встреча с подругой, размышления о жизни, поиск своего пути.

**Особенности игры:**
- Внутренняя битва между Разумом и Чувствами при принятии решений
- Мини-игра "Камень-ножницы-бумага" в ключевые моменты
- Сбор артефактов-воспоминаний о свиданиях
- Система сохранения с возвратом артефактов при отказе
- Три ветки с множеством концовок

**Цель:** Пройти все ветки, собрать коллекцию воспоминаний и найти свой идеальный финал.


## Импорт библиотек

Используются стандартные библиотеки Python:
- `random` — для случайных событий
- `os` - для работы с файлами
- `datetime` - для отметки времени в сохранениях

In [ ]:
import random
import os
from datetime import datetime

#Классы для внутренней битвы
В этом блоке описаны классы для внутреннего конфликта между Разумом и Чувствами.
Эти классы используются для механики принятия решений в игре.

Класс `Effect` — базовый класс для всех эффектов.
Он хранит название эффекта и количество ходов, сколько он действует.

Класс `DoubtEffect` — Наследуется от Effect. Эффект сомнений ослабляет уверенность персонажа.

Класс `InspirationEffect` — Наследуется от Effect. Эффект вдохновения усиливает следующий ход.

Класс `InnerVoice` — Базовый класс для внутренних голосов (Разума и Чувств). Хранит основные параметры и методы для боя.

Класс `Reason` — Наследуется от InnerVoice. Представляет внутренний голос Разума. Отвечает за логичные, рациональные решения.

Класс `Feelings` — Наследуется от InnerVoice. Представляет внутренний голос Чувств.Отвечает за эмоциональные, страстные решения.



In [ ]:
class Effect:
    def __init__(self, name, duration):
        self.name = name
        self.duration = duration

    def apply(self, target):
        pass


class DoubtEffect(Effect):
    def __init__(self):
        super().__init__("Сомнения", 2)

    def apply(self, target):
        damage = 15
        target.health = max(0, target.health - damage)
        print(f"🦠 {self.name} наносит {damage} урона")


class InspirationEffect(Effect):
    def __init__(self):
        super().__init__("Вдохновение", 1)

    def apply(self, target):
        print(f"✨ {self.name} - следующий ход будет усилен!")


class InnerVoice:
    def __init__(self, name, health):
        self.name = name
        self.health = health
        self.effects = []

    @property
    def is_alive(self):
        return self.health > 0

    def take_damage(self, damage):
        self.health = max(0, self.health - damage)
        print(f"💥 {self.name} получает {damage} урона | HP: {self.health}")

    def add_effect(self, effect):
        self.effects.append(effect)
        print(f"⚡ На {self.name} наложен эффект: {effect.name}")

    def process_effects(self):
        for effect in self.effects[:]:
            effect.apply(self)
            effect.duration -= 1
            if effect.duration <= 0:
                self.effects.remove(effect)
                print(f"❄️ Эффект {effect.name} закончился")


class Reason(InnerVoice):
    def __init__(self):
        super().__init__("Разум", 120)

    def attack(self, target):
        print("🧠 Атака логикой!")
        damage = random.randint(15, 25)
        target.take_damage(damage)
        if random.random() < 0.4:
            target.add_effect(DoubtEffect())


class Feelings(InnerVoice):
    def __init__(self):
        super().__init__("Чувства", 100)

    def attack(self, target):
        print("💖 Атака эмоциями!")
        damage = random.randint(10, 30)
        target.take_damage(damage)
        if random.random() < 0.3:
            target.add_effect(InspirationEffect())

#Файлы и начальные данные
Здесь задаются имена файлов, которые используются в игре.
В них хранятся пользователи и коллекция артефактов.

In [ ]:
users_file = "players.txt"
artifacts_file = "memories_vault.txt"

default_artifacts = [
    "📸 Фото с первого свидания",
    "🎟 Билет в кино",
    "💌 Любовное письмо",
    "🎁 Браслет дружбы",
    "🍫 Коробка конфет",
    "🎵 Микстейп с песнями",
    "🌹 Засушенная роза"
]

#Инициализация файлов игры
Функция для создания начальных файлов игры.
Вызывается при первом запуске.

In [ ]:
def initialize_files():
    if not os.path.exists(users_file):
        with open(users_file, "w", encoding="utf-8") as f:
            f.write("# Логин:Пароль\n")
            f.write("admin:admin123\n")
            f.write("test:test123\n")

    if not os.path.exists(artifacts_file):
        with open(artifacts_file, "w", encoding="utf-8") as f:
            for artifact in default_artifacts:
                f.write(artifact + "\n")

initialize_files()

## Загрузка и сохранение списков

Эти функции используются для работы с файлами, в которых данные хранятся построчно.

`load_list` читает файл и возвращает список строк.  
Если файл не найден, возвращается пустой список.

`save_list` записывает элементы списка в файл, по одному элементу в строке.


In [ ]:
def load_list(filename):
    try:
        with open(filename, "r", encoding="utf-8") as f:
            return [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        return []


def save_list(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for item in data:
            f.write(item + "\n")

# Имена файлов игрока

Эти функции формируют имена файлов для хранения данных конкретного пользователя.

Для каждого логина создаются свои файлы:
- один для состояния персонажа
- один для инвентаря

Так данные разных игроков не пересекаются.

In [ ]:
def player_save_file(login):
    return f"save_{login}.txt"


def player_artifacts_file(login):
    return f"artifacts_{login}.txt"

#Камень-ножницы-бумага
Мини-игра для взаимодействия с персонажами.

In [ ]:
def play_rock_paper_scissors(opponent_name, win_prize, lose_consequence):
    """
    Мини-игра "Камень-Ножницы-Бумага"
    """
    print(f"\n {opponent_name} предлагает сыграть в игру!")

    moves = {"1": "камень", "2": "ножницы", "3": "бумага"}

    print("\n Выберите вариант:")
    print("1.  Камень")
    print("2.  Ножницы")
    print("3.  Бумага")

    print(f"\n При победе: {win_prize}")
    print(f" При поражении: {lose_consequence}")

    used_choices = []

    while True:
        print("\n Ваш ход:")
        choice = input("Ваш выбор (1-3): ").strip()

        if choice not in moves:
            print(" Неверный ввод. Выберите 1, 2 или 3.")
            continue

        used_choices.append(choice)  # запоминаем выбор

        player_move = moves[choice]
        print(f"Вы выбрали: {player_move}")

        # 60% - детерминированный алгоритм
        # 40% - полностью случайный выбор
        if random.random() < 0.6:
            # Детерминированный алгоритм на основе длины использованных выборов
            choice_num = int(choice)
            opponent_index = (choice_num + len(used_choices)) % 3

            if opponent_index == 0:
                opponent_choice = "3"
            elif opponent_index == 1:
                opponent_choice = "1"
            else:
                opponent_choice = "2"
        else:
            # Случайный выбор
            opponent_choice = random.choice(["1", "2", "3"])

        opponent_move = moves[opponent_choice]
        print(f"{opponent_name} выбрал: {opponent_move}")

        if player_move == opponent_move:
            print(" Ничья! Играем еще раз!")
            continue

        if (player_move == "камень" and opponent_move == "ножницы") or \
           (player_move == "ножницы" and opponent_move == "бумага") or \
           (player_move == "бумага" and opponent_move == "камень"):
            print(f" Вы победили! {win_prize}")
            return True
        else:
            print(f" Вы проиграли! {lose_consequence}")
            return False

#Основные функции игры
Здесь реализована основная логика игры с тремя сюжетными ветками.

In [ ]:
def inner_battle():
    """Запуск внутренней битвы между Разумом и Чувствами"""
    print("⚔️ ВНУТРЕННЯЯ БИТВА: РАЗУМ vs ЧУВСТВА")

    # Создаем объекты Разума и Чувств
    reason = Reason()
    feelings = Feelings()

    turn = 0
    while reason.is_alive and feelings.is_alive:
        turn += 1
        print(f"\n Ход {turn}")
        print(f"🧠 Разум: {reason.health} HP | 💖 Чувства: {feelings.health} HP")

        # Ход Разума
        reason.process_effects()
        if reason.is_alive:
            reason.attack(feelings)

        if not feelings.is_alive:
            print("\n ПОБЕДИЛ РАЗУМ! Принято логичное решение.")
            print("'Иногда нужно слушать голову, а не сердце', - думаете вы.")
            return "reason"

        # Ход Чувств
        feelings.process_effects()
        if feelings.is_alive:
            feelings.attack(reason)

        if not reason.is_alive:
            print("\n ПОБЕДИЛИ ЧУВСТВА! Принято эмоциональное решение.")
            print("'Сердце не обманешь', - решаете вы, следуя эмоциям.")
            return "feelings"

    print("\n НИЧЬЯ! Принято компромиссное решение.")
    print("'Нужно найти золотую середину', - приходите вы к выводу.")
    return "compromise"

In [ ]:
def date_with_alex():
    """Ветка свидания с Алексом"""
    print("💙 ВЕТКА: АЛЕКС - НАДЕЖНЫЙ И ЗАБОТЛИВЫЙ")

    artifacts_gained = []

    print("\n Вы берете трубку, и слышите спокойный голос Алекса.")
    print(" 'Привет, Даш! Я тут подумал... Как насчет кофе в \"Уютном уголке\"?'")
    print(" Там открылась новая кондитерская, и я знаю, как ты любишь десерты.'")
    print("\nВы слышите в его голосе легкое волнение, но также и искреннюю заботу.")
    print("'Хорошо, Алекс, давай встретимся', - отвечаете вы.")

    print("\n Вы стоите перед зеркалом, выбирая наряд:")
    print("1. Элегантное черное платье - для особого случая")
    print("2. Уютные джинсы и любимый свитер - чтобы чувствовать себя комфортно")
    print("3. Спортивный костюм - вам сегодня лень выбирать")

    choice = input("\nВаш выбор (1-3): ").strip()

    if choice == "1":
        print("\n Вы приходите в платье. Алекс смотрит на вас, замирая на секунду.")
        print(" 'Ты... ты выглядишь прекрасно', - говорит он, немного смущаясь.")
        artifacts_gained.append("👗 Элегантное черное платье - в нем вы чувствовали себя особенной")
    elif choice == "2":
        print("\n Вы в уютном наряде. Алекс тепло улыбается.")
        print(" 'Как всегда, естественно и по-домашнему мило', - замечает он.")
    else:
        print("\n Вы в спортивном костюме. Алекс смотрит с легким удивлением.")
        print(" 'Главное, чтобы тебе было удобно', - говорит он дипломатично.")

    # Мини-игра с Алексом
    print("\n За кофе Алекс с хитрым прищуром говорит:")
    print(" 'Давай решим, кто платит, по-честному. Сыграем в камень-ножницы-бумага?'")
    print(" 'Если выиграешь - я плачу за оба заказа. Если проиграешь... ну, ты поняла.'")

    if play_rock_paper_scissors(
        opponent_name="Алекс",
        win_prize="Алекс платит за оба заказа",
        lose_consequence="Вы платите за кофе"
    ):
        print("\n Алекс с улыбкой достает кошелек.")
        print(" 'Честная игра, честная победа. Сегодня мое угощение!'")
        artifacts_gained.append("💳 Чек из 'Уютного уголка' - оплачен Алексом со словами: 'Мне приятно'")
    else:
        print("\n Вы с легкой досадой достаете кошелек.")
        print(" 'В следующий раз обязательно отыграюсь!', - смеется Алекс.")

    print("\n За столиком воцаряется комфортная тишина. О чем поговорить?")
    print("1. О работе и карьере - Алекс рассказывает о своих IT-проектах")
    print("2. О хобби и увлечениях - узнать друг друга лучше")
    print("3. О чувствах - рискнуть и поговорить начистоту")

    choice = input("\nВаш выбор (1-3): ").strip()

    if choice == "1":
        print("\n Алекс оживляется, рассказывая о своем новом проекте.")
        print(" 'Я разрабатываю приложение для планирования... Оно еще сырое, но...'")
        print("Вы слушаете его полчаса, видя, как горят его глаза.")
    elif choice == "2":
        print("\n 'Знаешь, я недавно начал рисовать акварелью', - неожиданно говорит Алекс.")
        print(" 'И еще... я играю на гитаре. Не очень хорошо, но для души.'")
        artifacts_gained.append("🎨 Акварельный набросок - подаренный Алексом на прощание")
    else:
        print("\n Алекс отводит взгляд, потом снова смотрит на вас.")
        print(" 'Даша, я давно хотел сказать... ты мне очень нравишься.'")
        print(" 'Я знаю, что ты, возможно, не чувствуешь того же...'")
        print(" 'Но я готов ждать. Столько, сколько потребуется.'")

    print("\n Свидание подходит к концу. За окном уже стемнело.")
    print(" 'Можно я провожу тебя до дома?' - предлагает Алекс.")
    print("\nВаши варианты:")
    print("1. 'Да, давай пройдемся. Луна сегодня красивая...'")
    print("2. 'Спасибо, но я, пожалуй, вызову такси. Устала немного.'")
    print("3. 'Давай встретимся завтра? В парке, например...'")

    choice = input("\nВаш выбор (1-3): ").strip()

    if choice == "1":
        print("\n Вы идете по тихим улицам. Алекс осторожно берет вас за руку.")
        print(" 'Знаешь, Даш, я понимаю, что ты не чувствуешь то же самое...'")
        print(" 'Но я буду рядом. Всегда. Когда будешь готова - я здесь.'")
        artifacts_gained.append("🌙 Прогулка под луной - его рука была теплой и надежной")

        # Финал ветки Алекса
        print("\n" + "="*60)
        print(" МЕСЯЦ СПУСТЯ...")
        print("="*60)
        print("\nАлекс был рядом все это время. Помогал с учебой, поддерживал,")
        print("слушал, когда нужно было выговориться. Его постоянство начало")
        print("постепенно менять что-то внутри вас.")
        print("\n 'Он же такой... надежный', - думаете вы.")
        print(" 'С ним спокойно. Безопасно. Как дома.'")

        print("\n ПЕРЕД ВАМИ ВЫБОР:")
        print("1. Принять его чувства - дать шанс этой тихой, теплой любви")
        print("2. Остаться друзьями - не рисковать тем, что уже есть")
        print("3. Взять паузу - нужно время, чтобы все обдумать")

        final_choice = input("\nВаш выбор (1-3): ").strip()

        if final_choice == "1":
            print("💍 КОНЦОВКА: 'ДРУЖБА, ПЕРЕРОСШАЯ В ЛЮБОВЬ'")
            print("\nВы смотрите на Алекса и понимаете - да, именно этого человека")
            print("вы хотите видеть рядом. Не страсть, не буря эмоций, а тихое,")
            print("стабильное счастье. То, что строится годами.")
            print("\n 'Алекс... Давай попробуем', - говорите вы тихо.")
            print("Его лицо озаряет такая улыбка, что у вас перехватывает дыхание.")
            print("\n 'Я ждал этого момента целый год', - шепчет он.")
            artifacts_gained.append("💍 Скромное серебряное кольцо - 'Не обещание, а надежда', - сказал Алекс")
            return artifacts_gained, "Дружба, переросшая в любовь - он ждал, и дождался"

        elif final_choice == "2":
            print("🤝 КОНЦОВКА: 'НАДЕЖНЫЕ ДРУЗЬЯ НАВСЕГДА'")
            print("\nВы смотрите на Алекса и понимаете - нет, это не любовь.")
            print("И будет нечестно давать ему надежду.")
            print("\n 'Алекс, ты лучший друг, который у меня есть...'")
            print(" '...и я не хочу это разрушать. Прости.'")
            print("\nОн кивает, в его глазах боль, но также понимание.")
            print(" 'Хорошо, Даш. Друзья так друзья. Я всегда буду рядом.'")
            return artifacts_gained, "Остались надежными друзьями - иногда дружба ценнее любви"

        else:
            print("⏸️ КОНЦОВКА: 'ВРЕМЯ НА РАЗМЫШЛЕНИЯ'")
            print("\n 'Дай мне время, Алекс. Я не могу ответить сейчас.'")
            print("Он кивает, пытаясь скрыть разочарование.")
            print("\n 'Сколько нужно, столько и жду. Я никуда не уйду.'")
            print("\nВы расстаетесь, и в воздухе висит вопрос без ответа...")
            return artifacts_gained, "Время на паузе - ответ где-то между 'да' и 'нет'"

    elif choice == "2":
        print("\n Вы вызываете такси и садитесь в машину.")
        print(" 'Спасибо за прекрасный вечер, Алекс', - говорите вы через окно.")
        print(" 'Это было... мило. Надеюсь увидеться снова', - отвечает он.")
        print("\nМашина трогается, и вы видите в зеркале заднего вида, как он")
        print("еще долго стоит на тротуаре, провожая вас взглядом.")
        return artifacts_gained, "Вежливое прощание - 'до свидания', но не 'прощай'"

    else:
        print("\n 'Давай встретимся завтра? В парке, в три?' - предлагаете вы.")
        print("Лицо Алекса озаряется улыбкой.")
        print(" 'Конечно! Я уже жду не дождусь!'")
        artifacts_gained.append("🗓️ Запись в ежедневнике - 'Завтра, 15:00, парк с Алексом'")
        return artifacts_gained, "Свидание продолжается завтра - история еще не закончена"

In [ ]:
def date_with_maxim():
    """Ветка свидания с Максимом"""
    print("🎸 ВЕТКА: МАКСИМ - СТРАСТНЫЙ И НЕПРЕДСКАЗУЕМЫЙ")

    artifacts_gained = []

    print("\n Сообщение от Максима заставляет ваше сердце биться чаще:")
    print(" 'Красотка! Концерт сегодня в 'Клубе 27'. Моя группа играет.'")
    print(" 'Будешь моей музой на вечере? Жду у сцены в 20:00. Не опаздывай! ;)'")
    print("\nВы чувствуете знакомое волнение. С Максимом никогда не скучно.")
    print("'Хорошо, буду', - отвечаете вы, уже предвкушая вечер.")

    print("\n Вы входите в клуб. Группа 'Максимальное напряжение' уже на сцене.")
    print("Максим в рваных джинсах и кожаной куртке, он полностью поглощен музыкой.")
    print("\n Как себя вести?")
    print("1. Подойти к самой сцене, танцевать, ловить его взгляд")
    print("2. Остаться у барной стойки, наблюдать со стороны")
    print("3. Уйти вглубь клуба, найти подруг")

    choice = input("\nВаш выбор (1-3): ").strip()

    if choice == "1":
        print("\n Вы пробираетесь к сцене. Максим замечает вас и улыбается.")
        print("Во время гитарного соло он смотрит прямо на вас.")
        print("Ваше сердце замирает на секунду.")
        artifacts_gained.append("🎫 Браслет с концерта - 'Клуб 27, только для своих'")
    elif choice == "2":
        print("\n Вы стоите у бара, наблюдая за ним. Он выглядит как рок-звезда:")
        print("вспотевшие волосы, сосредоточенное лицо, полная отдача музыке.")
        print("Вы не можете отвести глаз.")
    else:
        print("\n В глубине клуба вы встречаете подругу.")
        print(" 'Боже, Даш, ты же влюблена в него по уши!', - кричит она вам на ухо.")
        print("Вы только краснеете в ответ.")

    # Мини-игра с Максимом
    print("\n После концерта Максим, весь мокрый от пота, подходит к вам.")
    print(" 'Отлично сыграли, да? А теперь - честный баттл!'")
    print(" 'Если выиграешь - сыграю для тебя сольный концерт. Прямо здесь и сейчас.'")
    print(" 'Если проиграешь... споешь для моих друзей караоке!'")

    if play_rock_paper_scissors(
        opponent_name="Максим",
        win_prize="Сольный концерт только для вас",
        lose_consequence="Караоке для друзей Максима"
    ):
        print("\n Максим берет гитару и уводит вас в тихий угол.")
        print(" 'Только для тебя, муза моя', - шепчет он.")
        print("Он играет акустическую версию своей песни, глядя вам в глаза.")
        artifacts_gained.append("🎸 Сольный концерт - в его глазах горел особый огонь")
    else:
        print("\n Друзья Максима поднимают вас на сцену.")
        print(" 'Споем 'Беспечного ангела'!', - кричит кто-то из зала.")
        print("Вы краснеете, но подхватываете микрофон.")
        artifacts_gained.append("🎤 Караоке-микрофон - 'Ты классно поешь!', - сказал потом Максим")

    print("\n На послеконцертной вечеринке к Максиму постоянно подходят девушки.")
    print("Он флиртует, смеется, принимает комплименты.")
    print("\nВаша реакция?")
    print("1. Ревновать и показывать это - нахмуриться, отойти в сторону")
    print("2. Игнорировать - веселиться с друзьями отдельно")
    print("3. Уйти, не прощаясь - пусть ищет потом")

    choice = input("\nВаш выбор (1-3): ").strip()

    if choice == "1":
        print("\n Вы демонстративно хмуритесь. Максим замечает это и подходит.")
        print(" 'Ревнуешь?', - спрашивает он с хитрой улыбкой.")
        print(" 'Это мило. Но не переживай, ты сегодня - моя муза.'")
    elif choice == "2":
        print("\n Вы танцуете с друзьями, делая вид, что не замечаете.")
        print("Максим наблюдает за вами с интересом через всю комнату.")
        print("В его взгляде - любопытство и одобрение.")
    else:
        print("\n Вы собираете вещи и уходите, не попрощавшись.")
        print("Через час телефон разрывается от звонков.")
        print(" 'Где ты? Я ищу тебя по всему клубу!', - голос Максима в трубке.")
        artifacts_gained.append("📞 Скриншот пропущенного звонка - 17 пропущенных от Максима")

    print("\n Поздней ночью вы с Максимом выходите на пожарную лестницу.")
    print("Городские огни мерцают внизу. Он закуривает, глядя вдаль.")
    print("\n 'Даш, ты не такая, как другие', - начинает он.")
    print(" 'Ты... настоящая. Не играешь роли.'")
    print(" 'Но я должен быть честен...'")
    print(" '...я не готов к серьезным отношениям. Я музыкант, я свободен.'")
    print(" 'Я живу сегодняшним днем. Завтра может не быть.'")

    print("\n Ваш ответ:")
    print("1. 'Я понимаю. Давай просто наслаждаться моментом'")
    print("2. 'Я изменю тебя. Сделаю серьезным'")
    print("3. 'Тогда это конец. Я ухожу сейчас'")

    choice = input("\nВаш выбор (1-3): ").strip()

    if choice == "1":
        print("\n Вы начинаете отношения без обязательств.")
        print("Это как американские горки: страстные ночи, ссоры на рассвете,")
        print("примирения с цветами и стихами. Вы живете от встречи до встречи.")

        # Финал ветки Максима
        print("🎬 ПОЛГОДА СПУСТЯ...")
        print("\nВы устали. Устали от непредсказуемости, от звонков в три ночи,")
        print("от его внезапных отъездов 'на творчество', от постоянной тревоги.")
        print("Максим то появляется с романтичными сюрпризами,")
        print("то исчезает на недели без объяснений.")

        print("\n ПЕРЕД ВАМИ ВЫБОР:")
        print("1. Разорвать эти токсичные отношения - хватит, пора заботиться о себе")
        print("2. Дать еще один шанс - а вдруг он изменится?")
        print("3. Изменить себя - подстроиться под его ритм жизни")

        final_choice = input("\nВаш выбор (1-3): ").strip()

        if final_choice == "1":
            print("💥 КОНЦОВКА: 'СГОРЕЛИ В СТРАСТИ И РАССТАЛИСЬ'")
            print("\nВы набираетесь смелости и говорите Максиму, что все кончено.")
            print(" 'Я устала, Макс. Устала ждать, устала гадать, устала бояться.'")
            print("\nОн сначала злится, потом умоляет остаться, наконец - принимает.")
            print(" 'Ты права. Я не заслуживаю тебя. Прости.'")
            print("\nЭто больно, как отрывание части себя. Но необходимо.")
            artifacts_gained.append("🔥 Сгоревшие письма Максиму - 'Больше не буду писать'")
            return artifacts_gained, "Сгорели в страсти и расстались - иногда любовь сжигает дотла"

        elif final_choice == "2":
            print("🌀 КОНЦОВКА: 'БЕСКОНЕЧНЫЕ КАЧЕЛИ ЭМОЦИЙ'")
            print("\nВы остаетесь. Цикл повторяется: страсть-боль-прощение-страсть.")
            print("Годы проходят, а ничего не меняется.")
            print("Вы становитесь тенью себя, живущей ожиданиями его звонка.")
            return artifacts_gained, "Бесконечные эмоциональные качели - день за днем, год за годом"

        else:
            print("🎭 КОНЦОВКА: 'ПОТЕРЯ СЕБЯ В ЛЮБВИ'")
            print("\nВы пытаетесь стать 'идеальной' для него.")
            print("Бросаете учебу, чтобы ездить с ним на гастроли.")
            print("Меняете друзей, привычки, даже стиль одежды.")
            print("\nНо теряете себя в процессе. Вы больше не Даша,")
            print("а тень человека, живущая отрывками его внимания.")
            return artifacts_gained, "Потеря себя в отношениях - заплатила слишком высокую цену"

    elif choice == "2":
        print("\n 'Я сделаю тебя серьезным', - говорите вы с уверенностью.")
        print(" 'Я научу тебя любить по-настоящему.'")
        print("\nМаксим смеется, но в его глазах - интерес.")
        print(" 'Ну что ж, попробуй. Но это будет непросто.'")

        # Финал для выбора "изменить его"
        print("🎬 ГОД СПУСТЯ...")
        print("\nВы пытаетесь 'перевоспитать' Максима. Это выматывает.")
        print("Он сопротивляется, срывается на гастроли, возвращается с извинениями.")
        print("\n 'Даша, я пытаюсь, правда', - говорит он в сотый раз.")
        print(" 'Но я музыкант. Моя жизнь - это сцена, дорога, свобода.'")
        print("\nОднажды вы просыпаетесь и понимаете: вы не меняете его.")
        print("Он меняет вас. Вы стали нервной, подозрительной, уставшей.")
        print("\n 'Хватит', - говорите вы себе. 'Это не любовь. Это миссия.'")
        print("И вы отпускаете его. Не потому что перестали любить.")
        print("А потому что полюбили себя больше.")

        return artifacts_gained, "Миссия по изменению Максима - пыталась изменить того, кто не хотел меняться"

    else:
        print("\n Вы разворачиваетесь и уходите без слов.")
        print("Максим не бежит за вами. Он просто смотрит вам вслед, затягиваясь сигаретой.")
        print("Это конец. Чистый, ясный, болезненный.")
        print("\nВы не плачете. Вы чувствуете странное облегчение.")
        print("Как будто сняли тяжелый рюкзак, который тащили слишком долго.")

        artifacts_gained.append("Дверь, закрытая навсегда - больше не будет 'завтра'")
        return artifacts_gained, "Чистый разрыв - ушла, не оглядываясь"

In [ ]:
def date_with_yourself():
    """Ветка вечера наедине с собой"""
    print("🌟 ВЕТКА: ВСТРЕЧА С СОБОЙ")

    artifacts_gained = []

    print("\n Вы отключаете телефон и остаетесь дома.")
    print("Тишина. Непривычная, но приятная тишина.")
    print("Впервые за долгое время у вас есть время только для себя.")
    print("\n 'А что, если сегодняшний вечер посвятить только себе?'")
    print("'Без Алекса, без Максима. Только я и мои мысли.'")

    print("\n Чем заняться в этот особенный вечер?")
    print("1. Принять долгую ванну с пеной - с книгой и свечами")
    print("2. Пересмотреть старые фото и дневники - вспомнить себя")
    print("3. Заняться творчеством - рисовать, писать, творить")

    choice = input("\nВаш выбор (1-3): ").strip()

    if choice == "1":
        print("\n Вы наполняете ванну, добавляете ароматную пену с лавандой.")
        print("Теплая вода обволакивает, снимая напряжение последних недель.")
        print("Вы берете книгу, но через пять минут просто закрываете глаза.")
        print(" 'Когда я в последний раз просто... была? Не думала, не решала?'")
        artifacts_gained.append("🛁 Соль для ванны с лавандой - пахнет спокойствием и тишиной")

    elif choice == "2":
        print("\n Вы достаете с антресолей старую коробку с фотографиями.")
        print("Вот вы в школе - смешная, с косичками. Вот первый выпускной.")
        print("Вот фото с подругами, которых не видели уже годы.")
        print("\n 'Куда делась та девчонка? Та, что смеялась громче всех?'")
        artifacts_gained.append("📖 Страница из старого дневника - 'Хочу стать счастливой. Просто счастливой.'")

    else:
        print("\n Вы достаете краски и холст. Не планируете шедевр - просто рисуете.")
        print("Цвета смешиваются, формы рождаются сами собой.")
        print("Это абстракция, но она выражает то, что нельзя сказать словами.")
        print("Боль, надежду, сомнения, мечты - все на одном холсте.")
        artifacts_gained.append("🖼️ Картина 'Вечер одиночества' - синие тоска, желтые надежды, красные страсти")

    print("\n Мысли сами возвращаются к Алексу и Максиму...")
    print(" 'Алекс такой надежный... С ним было бы спокойно.'")
    print(" 'Но Максим... С ним так ярко. Так... живо.'")
    print(" 'А может, мне сейчас не нужен никто из них?'")
    print(" 'Может, нужна я сама? Та, которую я давно не слушаю?'")

    # Внутренняя битва
    battle_result = inner_battle()

    if battle_result == "reason":
        print("\n 'Мне нужна стабильность. Я устала от эмоциональных качелей.'")
        print(" 'Алекс мог бы дать мне это. Спокойствие. Надежность.'")
    elif battle_result == "feelings":
        print("\n 'Но как забыть эти бабочки в животе, когда вижу Максима?'")
        print(" 'Эта страсть, это безумие... Разве не ради этого живут?'")
    else:
        print("\n 'А может, правда - никого? Может, сначала найти себя,'")
        print(" 'а потом уже решать, кто достоин быть рядом?'")

    # Мини-игра с самой собой
    print("\n Вы решаете бросить себе вызов.")
    print(" 'Если выиграю у самой себя - с завтрашнего дня начинаю новую жизнь!'")
    print(" 'Если проиграю... оставлю все как есть. Буду выбирать между ними.'")

    if play_rock_paper_scissors(
        opponent_name="Свое второе Я",
        win_prize="Начать изменения с завтрашнего дня",
        lose_consequence="Оставить все как есть"
    ):
        print("\n 'Завтра начинается новая жизнь!', - решаете вы.")
        print(" 'Сначала я, потом - все остальное.'")
        artifacts_gained.append(" План изменений - список из 10 пунктов 'Что я хочу для себя'")
    else:
        print("\n 'Может, и не стоит ничего менять...', - вздыхаете вы.")
        print(" 'Выберу кого-то из них. Пусть решают за меня.'")
        artifacts_gained.append("🛋️ Уютный плед - 'Иногда проще укрыться, чем измениться'")

    print("\n Внезапный звонок заставляет вас вздрогнуть.")
    print("Незнакомый номер... Или нет? Голос в трубке знакомый:")
    print("'Привет, Даш! Это Катя, помнишь? Школьная подруга!'")
    print(" 'Я в городе проездом, на один вечер. Не хочешь встретиться?'")
    print("\n 'Катя... Та самая, с которой мы мечтали объездить весь мир.'")
    print(" 'Которая уехала в другой город и пропала на три года.'")

    print("\n Встречаться с Катей?")
    print("1. 'Да, почему нет? Старое-доброе, как раз кстати'")
    print("2. 'Нет, хочется побыть одной. Сегодня - только для себя'")
    print("3. 'Пригласить ее к себе - пусть расскажет все новости'")

    choice = input("\nВаш выбор (1-3): ").strip()

    if choice == "1":
        print("\n Вы встречаетесь в круглосуточном кафе 'У Бессонницы'.")
        print("Катя загорелая, с новыми морщинками у глаз, но с тем же огнем.")
        print(" 'Я развелась год назад', - говорит она без предисловий.")
        print(" 'И знаешь что? Это лучшее, что со мной случалось.'")
        print("\n 'Я нашла СЕБЯ. Не жену, не мать, не дочь - СЕБЯ.'")
        print(" 'Путешествую одна. Работаю удаленно. Живу, как хочу.'")
        artifacts_gained.append(" Чек из кафе 'Откровения' - на салфетке номер Кати: 'Звони, если захочешь перемен'")

        # Финал ветки "себя"
        print("\n" + "="*60)
        print(" РАЗГОВОР С КАТЕЙ ЗАСТАВЛЯЕТ ЗАДУМАТЬСЯ...")
        print("="*60)
        print("\Вы возвращаетесь домой под утро. Голова гудит от мыслей.")
        print(" 'Неужели я ищу счастье в других, вместо того чтобы найти его внутри?'")
        print(" 'Алекс, Максим... А где же Я в этом уравнении?'")

        print("\n ПЕРЕД ВАМИ ВЫБОР:")
        print("1. Взять паузу от отношений - год только для себя, как у Кати")
        print("2. Продолжить поиски 'того самого' - но теперь с четкими критериями")
        print("3. Уехать в путешествие - сменить обстановку, найти новые ответы")

        final_choice = input("\nВаш выбор (1-3): ").strip()

        if final_choice == "1":
            print("🌱 КОНЦОВКА: 'НАЧАЛО НОВОЙ ЖИЗНИ'")
            print("\nВы записываетесь на курсы фотографии, о которых давно мечтали.")
            print("Начинаете бегать по утрам. Читаете те книги, что откладывали 'на потом'.")
            print("\nМесяц за месяцем вы становитесь сильнее, увереннее, счастливее.")
            print("Без Алекса. Без Максима. Без чьей-либо помощи.")
            print("\n💭 'Я - это я. И этого достаточно.'")
            artifacts_gained.append("📚 Диплом с курсов саморазвития - 'Сначала полюби себя, потом - других'")
            return artifacts_gained, "Начало новой жизни с собой - самый важный роман в жизни"

        elif final_choice == "2":
            print("🔍 КОНЦОВКА: 'ПОИСКИ С ПРИНЦИПАМИ'")
            print("\nВы решаете, что все-таки хотите отношений.")
            print("Но теперь у вас есть список: 'Что я хочу в партнере'.")
            print("И первое правило: 'Он должен принимать меня настоящую.'")
            print("\n💭 'Не я под него, не он под меня. Мы - вместе, но каждый - себя.'")
            return artifacts_gained, "Поиски настоящих отношений - с новыми правилами и самоуважением"

        else:
            print("✈️ КОНЦОВКА: 'ДОРОГА ЗОВЕТ К ПЕРЕМЕНАМ'")
            print("\nВы покупаете билет в один конец. Бали, говорят, хорош в это время года.")
            print("Новая страна, новые люди, новые возможности.")
            print("\n💭 'Иногда, чтобы найти себя, нужно потеряться в мире.'")
            print(" 'А вернусь я другой. Или не вернусь вовсе.'")
            artifacts_gained.append(" Билет в неизвестность - 'Куда глаза глядят. Главное - вперед.'")
            return artifacts_gained, "Дорога зовет к переменам - иногда нужно уехать, чтобы вернуться"

    elif choice == "2":
        print("\n 'Извини, Кать, сегодня не могу', - говорите вы.")
        print(" 'Как-нибудь в другой раз. Сегодня я... занята.'")
        print("\nВы отключаете телефон и остаетесь наедине с мыслями.")
        print("Этот вечер становится переломным моментом...")
        print("Хотя вы пока не знаете, в какую сторону повернется ваша жизнь.")
        return artifacts_gained, "Вечер глубоких размышлений - тишина говорит громче слов"

    else:
        print("\n 'Приходи ко мне', - говорите вы. 'Буду ждать.'")
        print("Катя приходит с бутылкой вина и историями на всю ночь.")
        print("Вы говорите до утра. Смеетесь, плачете, вспоминаете.")
        print("\n'Даша, ты сильная', - говорит Катя на прощание.")
        print(" 'Не позволяй никому решать за тебя. Даже себе.'")
        print("\nОна ночует на диване. Утром уезжает, оставив после себя")
        print("ощущение, что мир больше, чем ваш город и два парня.")
        artifacts_gained.append("🛋️ Подушка с дивана для гостей - 'На ней спала Катя, принесшая ветер перемен'")
        return artifacts_gained, "Ночные откровения с подругой - иногда чужие истории меняют наши"

#Система сохранения и загрузки
Функции для управления прогрессом игрока.
Если игрок не сохраняется, прогресс теряется.

In [ ]:
def save_game_progress(login, artifacts, ending):
    """Сохранение игры с записью в файл"""
    # Формируем данные для сохранения
    save_data = {
        "ending": ending,
        "artifacts": ",".join(artifacts),
        "date": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

    # Сохраняем в файл игрока
    with open(player_save_file(login), "w", encoding="utf-8") as f:
        for key, value in save_data.items():
            f.write(f"{key}:{value}\n")

    # Сохраняем артефакты в отдельный файл
    with open(player_artifacts_file(login), "w", encoding="utf-8") as f:
        for artifact in artifacts:
            f.write(artifact + "\n")

    # Проверка на полную коллекцию
    vault = load_list(artifacts_file)
    if not vault:
        print("🎊 ПОЗДРАВЛЯЕМ! ВЫ СОБРАЛИ ВСЕ АРТЕФАКТЫ!")
        print("\n Запускается процедура генерации новых артефактов...")

        # Создаем новую коллекцию
        new_artifacts = [
            "📖 Совместный дневник - 'Наши истории на одной странице'",
            "🎨 Картина, нарисованная вместе - четыре руки, одно сердце",
            "☕ Пара кружек - 'Его' и 'Ее', но одинаково любимые",
            "🧣 Шарф, связанный вручную - каждый узелок - мысль о тебе",
            "🎭 Театральные билеты - 'Гамлет', и мы плакали оба",
            "🏖️ Песок с пляжа в бутылочке - память о нашем море",
            "💫 Звезда, названная в вашу честь - 'Теперь мы всегда на небе'"
        ]
        save_list(artifacts_file, new_artifacts)

        print(" Создан новый набор воспоминаний! Теперь в копилке 7 новых артефактов.")


def load_game_progress(login):
    """Загрузка сохраненной игры"""
    try:
        with open(player_save_file(login), "r", encoding="utf-8") as f:
            data = {}
            for line in f:
                line = line.strip()
                if line and ":" in line:
                    key, value = line.split(":", 1)
                    data[key] = value

            if "artifacts" in data:
                return data["artifacts"].split(",")
    except FileNotFoundError:
        pass
    return []


def ask_to_save(login, artifacts, ending):
    """Запрос на сохранение игры с возможностью отказа"""
    print("💾 СОХРАНЕНИЕ ИГРЫ")

    print(f"\n Игрок: {login}")
    print(f" Собрано артефактов: {len(artifacts)}")
    print(f" Полученная концовка: {ending}")

    if artifacts:
        print("\n Ваши воспоминания с этого свидания:")
        for i, item in enumerate(artifacts, 1):
            print(f"  {i}. {item}")

    print("⚠️ ВАЖНО: Если не сохраните, прогресс сгорит!")
    print("   Все артефакты вернутся в общую копилку.")

    print("\n Что хотите сделать?")
    print("1. Сохранить игру - ваши артефакты останутся с вами")
    print("2. Не сохранять - все вернется в начало")

    choice = input("\nВаш выбор (1-2): ").strip()

    if choice == "1":
        save_game_progress(login, artifacts, ending)
        print("\n✅ Игра сохранена успешно!")
        print("💫 Ваши воспоминания в безопасности. Можете продолжить в любое время.")
        return True
    else:
        # Возвращаем артефакты в копилку (по ТЗ)
        if artifacts:
            vault = load_list(artifacts_file)
            vault.extend(artifacts)
            save_list(artifacts_file, vault)
            print(f"\n Прогресс не сохранен.")
            print(f" {len(artifacts)} артефактов возвращены в общую копилку.")
            print(" Все ваши выборы и концовка потеряны.")
        else:
            print("\n Прогресс не сохранен.")
            print(" Вы не собрали артефактов, так что терять нечего.")
        return False

#Главное меню и запуск игры

Основной цикл игры с меню выбора.

In [ ]:
def show_player_status(login):
    """Показать статус текущего игрока"""
    print("📊 ВАШ ТЕКУЩИЙ СТАТУС")

    # Проверяем сохранения
    save_file = player_save_file(login)
    if os.path.exists(save_file):
        with open(save_file, "r", encoding="utf-8") as f:
            lines = f.readlines()
            if lines:
                print(f"👤 Игрок: {login}")
                for line in lines:
                    if line.startswith("ending:"):
                        ending = line.split(":", 1)[1].strip()
                        print(f"📜 Последняя концовка: {ending}")
                    elif line.startswith("date:"):
                        date = line.split(":", 1)[1].strip()
                        print(f"📅 Дата сохранения: {date}")
    else:
        print(f"👤 Игрок: {login}")
        print(" Сохранений нет - начните новую игру!")

    # Проверяем артефакты игрока
    artifacts_file = player_artifacts_file(login)
    if os.path.exists(artifacts_file):
        with open(artifacts_file, "r", encoding="utf-8") as f:
            artifacts = [line.strip() for line in f if line.strip()]
            print(f"🎁 Ваших артефактов: {len(artifacts)}")
    else:
        print("🎁 Ваших артефактов: 0")


def main():
    """Главная функция игры"""
    print("\n" + "="*60)
    print("           🎮 СЕРДЕЧНЫЙ ПЕРЕКРЕСТОК 🎮")
    print("="*60)
    print("         История о выборе, любви и встрече с собой")
    print("="*60)

    current_player = None

    while True:
        print("📋 ГЛАВНОЕ МЕНЮ")

        if current_player:
            show_player_status(current_player)
        else:
            print("👤 Вы не вошли в игру")

        print("\nВыберите действие:")
        print("1. 🔐 Войти в игру (логин: admin, пароль: admin123)")
        print("2. 📝 Зарегистрироваться")
        print("3. 🎮 Начать новую игру")
        print("4. 📊 Показать статистику копилки")
        print("5. 🚪 Выйти из игры")

        choice = input("\nВаш выбор (1-5): ").strip()

        if choice == "1":
            # Вход существующего пользователя
            print("🔐 ВХОД В ИГРУ")

            users = {}
            for line in load_list(users_file):
                if ":" in line and not line.startswith("#"):
                    user, pwd = line.split(":", 1)
                    users[user] = pwd

            if not users:
                print(" Нет зарегистрированных игроков. Пожалуйста, зарегистрируйтесь.")
                continue

            username = input("Введите логин: ").strip()
            password = input("Введите пароль: ").strip()

            if username in users and users[username] == password:
                current_player = username
                print(f"\n Добро пожаловать, {username}!")
                print(" Ваш мир выборов и воспоминаний ждет вас...")
            else:
                print("\n Неверный логин или пароль.")
                print(" Попробуйте: admin / admin123")

        elif choice == "2":
            # Регистрация нового пользователя
            print("📝 РЕГИСТРАЦИЯ НОВОГО ИГРОКА")

            users = {}
            for line in load_list(users_file):
                if ":" in line and not line.startswith("#"):
                    user, pwd = line.split(":", 1)
                    users[user] = pwd

            username = input("Придумайте логин: ").strip()
            if not username:
                print("❌ Логин не может быть пустым.")
                continue

            if username in users:
                print(f"❌ Игрок с логином '{username}' уже существует.")
                continue

            password = input("Придумайте пароль: ").strip()
            if not password:
                print("❌ Пароль не может быть пустым.")
                continue

            # Сохраняем нового пользователя
            with open(users_file, "a", encoding="utf-8") as f:
                f.write(f"{username}:{password}\n")

            current_player = username
            print(f"\n✅ Пользователь '{username}' успешно создан!")
            print(" Теперь вы можете начать свою историю.")

        elif choice == "3":
            # Начать новую игру
            if not current_player:
                print("\n❌ Сначала войдите или зарегистрируйтесь.")
                print("Используйте вариант 1 или 2 из меню.")
                continue

            print("🎬 НАЧАЛО НОВОЙ ИСТОРИИ")

            print("\n Пятничный вечер. Вы сидите дома, когда...")
            print("\n *Вибрация телефона*")
            print(" Алекс: 'Привет, Даш! Как насчёт кофе в 'Уютном уголке'?")
            print("   'У них новый десерт - твой любимый шоколадный торт.'")
            print("   'Если хочешь, можем просто поболтать. Я соскучился.'")

            print("\n *Еще одна вибрация*")
            print("    Максим: 'Эй, красотка! Концерт сегодня в 'Клубе 27'.")
            print("   'Моя группа в главном сет-листе. Будешь моей музой?'")
            print("   'После - вечеринка за кулисами. Скучно не будет! ;)'")

            print("\n *Внутренний голос*")
            print(" 'А может... остаться дома? Принять ванну, почитать книгу.'")
            print("   'Просто побыть наедине с собой. Без выбора, без суеты.'")
            print("   'Иногда самый важный выбор - это выбор в пользу себя.'")

            print(" ВАШ ВЫБОР НА ЭТОТ ВЕЧЕР")
            print("1. 💙 Пойти на кофе с Алексом - спокойствие, надежность, тихое счастье")
            print("2. 🎸 На концерт к Максиму - страсть, адреналин, жизнь на полную")
            print("3. 🌟 Остаться дома одной - встреча с собой, разговор по душам")

            branch_choice = input("\nКуда отправитесь? (1-3): ").strip()

            if branch_choice == "1":
                print("\n Вы набираете номер Алекса...")
                artifacts, ending = date_with_alex()
            elif branch_choice == "2":
                print("\n Вы пишете Максиму 'Буду!'...")
                artifacts, ending = date_with_maxim()
            else:
                print("\n Вы выключаете телефон и укутываетесь в плед...")
                artifacts, ending = date_with_yourself()

            # Предложение сохранить игру
            saved = ask_to_save(current_player, artifacts, ending)

            if saved:
                print("🔄 ЧТО ДАЛЬШЕ?")
                print("1.  Сыграть еще раз - попробовать другую ветку")
                print("2.  Вернуться в меню - посмотреть статистику")

                replay = input("\nВаш выбор (1-2): ").strip()
                if replay != "1":
                    continue
            else:
                print("\n🔄 Возвращаюсь в главное меню...")
                continue

        elif choice == "4":
            # Показать статистику копилки
            print("📈 СТАТИСТИКА КОПИЛКИ ВОСПОМИНАНИЙ")

            vault = load_list(artifacts_file)
            print(f"🎁 Всего артефактов в копилке: {len(vault)}")

            if vault:
                print("\n Содержимое копилки:")
                for i, artifact in enumerate(vault, 1):
                    print(f"  {i}. {artifact}")
            else:
                print("\n Копилка пуста! Соберите артефакты в игре.")

            # Показать файлы игроков
            print("\n👥 Зарегистрированные игроки:")
            users = load_list(users_file)
            player_count = 0
            for line in users:
                if ":" in line and not line.startswith("#"):
                    player_count += 1
            print(f"  Всего игроков: {player_count}")

        elif choice == "5":
            # Выход из игры
            print("👋 ДО НОВЫХ ВСТРЕЧ!")
            3
            21
            print("\n💫 Спасибо, что играли в 'Сердечный Перекресток'.")
            print("🌟 Помните: каждый выбор - это новая история.")
            print("❤️ И самая важная история - это история с собой.")
            break

        else:
            print("\n❌ Неверный выбор. Пожалуйста, выберите от 1 до 5.")


# Запуск игры
if __name__ == "__main__":
    main()


           🎮 СЕРДЕЧНЫЙ ПЕРЕКРЕСТОК 🎮
         История о выборе, любви и встрече с собой
📋 ГЛАВНОЕ МЕНЮ
👤 Вы не вошли в игру

Выберите действие:
1. 🔐 Войти в игру (логин: admin, пароль: admin123)
2. 📝 Зарегистрироваться
3. 🎮 Начать новую игру
4. 📊 Показать статистику копилки
5. 🚪 Выйти из игры

Ваш выбор (1-5): 1
🔐 ВХОД В ИГРУ
Введите логин: 1
Введите пароль: 1

 Добро пожаловать, 1!
 Ваш мир выборов и воспоминаний ждет вас...
📋 ГЛАВНОЕ МЕНЮ
📊 ВАШ ТЕКУЩИЙ СТАТУС
👤 Игрок: 1
 Сохранений нет - начните новую игру!
🎁 Ваших артефактов: 0

Выберите действие:
1. 🔐 Войти в игру (логин: admin, пароль: admin123)
2. 📝 Зарегистрироваться
3. 🎮 Начать новую игру
4. 📊 Показать статистику копилки
5. 🚪 Выйти из игры

Ваш выбор (1-5): 3
🎬 НАЧАЛО НОВОЙ ИСТОРИИ

 Пятничный вечер. Вы сидите дома, когда...

 *Вибрация телефона*
 Алекс: 'Привет, Даш! Как насчёт кофе в 'Уютном уголке'?
   'У них новый десерт - твой любимый шоколадный торт.'
   'Если хочешь, можем просто поболтать. Я соскучился.'

 *Еще одна ви

KeyboardInterrupt: Interrupted by user